In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use("seaborn-darkgrid")

# 设置matplotlib正常显示中文和负号
# matplotlib.rcParams['font.sans-serif']=['SimHei']   # 用黑体显示中文
# matplotlib.rcParams['axes.unicode_minus']=False     # 正常显示负号


## Table

In [2]:
methods = ['BE1', 'BE2', 'BE3', 'BN', 'BP', 'RLHH']
# methods = ['BestEdges1', 'BestEdges2','BestEdges3','BestNodes','BestPaths','RLHH']

In [3]:
def concat_vertical(RLHH_file, baseline_filename):
    h = pd.read_csv(f"../result/baseline/{baseline_filename}.csv")
    hh = pd.read_csv(f"../result/{RLHH_file}.csv")
    hh['method'] = "RLHH"

    # 只比较 RLHH 中有的案例
    h = pd.merge(h, hh[['instance', 'n']], how='inner', on=['instance', 'n'])
    hh = pd.merge(hh, h[['instance', 'n']].drop_duplicates(), how='inner', on=['instance', 'n'])

    df = pd.concat([h, hh])
    df = df[['instance', 'n', 'method', 'iters', 'driver', 'objval', 'time']]
    df = df.sort_values(by=['n', 'instance', 'method'], ascending=True)
#     df['objval'] = df['objval'].apply(lambda x: round(x, 1))
#     df['time'] = df['time'].apply(lambda x: round(x, 2))
   
    return df[df['method'] != 'Hyper'].reset_index(drop=True)

In [27]:
def concat_horizon(df, columns, indexes=["n"]):
    index = df.loc[df["method"] == "BestEdges1", indexes].values
    h1   = df.loc[df["method"] == "BestEdges1", columns]
    h2   = df.loc[df["method"] == "BestEdges2", columns]
    h3   = df.loc[df["method"] == "BestEdges3", columns]
    h4   = df.loc[df["method"] == "BestNodes" , columns]
    h5   = df.loc[df["method"] == "BestPaths" , columns]
    rlhh = df.loc[df["method"] == "RLHH"      , columns]
    if len(columns) == 1:
        df_columns = indexes + ['BE1', 'BE2', 'BE3', 'BN', 'BP', 'RLHH']
    else:
        df_columns = [
            [''] * len(indexes) + ['BE1', 'BE1', 'BE2', 'BE2', 'BE3', 'BE3', 'BN', 'BN', 'BP', 'BP', 'RLHH', 'RLHH'], 
            indexes + ["obj", "time"]*6]
    df = pd.DataFrame(
        np.concatenate([index, h1, h2, h3, h4, h5, rlhh], axis=1), 
        columns=df_columns)
    return df

In [5]:
df_all = concat_vertical(RLHH_file="RLHH_all", baseline_filename="baseline_all")
# df_all.to_csv("../result/paper/all.csv", index=False)

In [6]:
def cal_best_method(df):
    df_value = df[methods].astype('int')
    df['rank'] = df_value.apply(lambda x: x.sort_values().index.to_numpy().argsort()[-1]+1, axis=1)
    df['min_idx'] = df_value.idxmin(axis=1)
    df['min_val'] = df_value.min(axis=1)
    columns = list(df)
    columns.insert(-3, columns.pop(columns.index('RLHH')))
    return df.loc[:, columns]

### 明细数据

In [7]:
df_all = pd.read_csv("../result/paper/all.csv")

In [8]:
df_objval = concat_horizon(df_all, "objval", indexes=["instance", "n"])
df_driver = concat_horizon(df_all, "driver", indexes=['instance', 'n'])
df_driver = cal_best_method(df_driver)
df_objval['min_driver'] = df_driver['min_val']
df_objval

,instance,n,BE1,BE2,BE3,BN,BP,RLHH,min_driver
0,50_01,50,26063,22834,25542,26939,23974,24096,13
1,50_02,50,26309,27970,25873,28821,26309,29115,15
2,50_03,50,21672,20456,20446,23041,23171,20474,11
3,50_04,50,25580,22785,24280,22650,28382,22899,13
4,50_05,50,29048,27713,28180,30657,29048,29277,16
...,...,...,...,...,...,...,...,...,...
105,200_06,200,206424,144500,265620,138720,206424,108412,65
106,200_07,200,208512,147242,267743,161212,235909,108816,64
107,200_08,200,233475,157020,168603,151429,208981,117852,70
108,200_09,200,242845,110578,137730,119793,172441,97628,56


In [9]:
df_objval[methods] = df_objval.apply(lambda x: x[methods] - 1440*x['min_driver'], axis=1)
# df_objval = cal_best_method(df_objval)
df_objval.drop(columns=['min_driver'], inplace=True)

In [10]:
# df_objval.to_csv("../result/paper/objval.csv", index=False)

### 数据处理

In [11]:
def df_stack(df):
    df = df.set_index(['instance', 'n']) #先把 instance ,n 设为索引
    df = df.stack() # 将列转化为二级索引
    df.index = df.index.rename('method', level=2) # 二级索引命命
    df.name = 'objval'
    df = df.reset_index() #将索引转化为Series
    return df

In [12]:
df_all = df_stack(df_objval)
df_all

,instance,n,method,objval
0,50_01,50,BE1,7343
1,50_01,50,BE2,4114
2,50_01,50,BE3,6822
3,50_01,50,BN,8219
4,50_01,50,BP,5254
...,...,...,...,...
655,200_10,200,BE2,46590
656,200_10,200,BE3,94876
657,200_10,200,BN,44214
658,200_10,200,BP,39474


### 平均 ObjVal

In [13]:
df_all = pd.read_csv('../result/paper/all_objval.csv')

In [14]:
mean_obj = df_all[['n', 'method', 'objval']].groupby(by=['n', 'method'], as_index=False).mean()
mean_obj = concat_horizon(mean_obj, 'objval', indexes=['n'])
mean_obj = cal_best_method(mean_obj)
mean_obj['n'] = mean_obj['n'].astype('int')
mean_obj[methods] = mean_obj[methods].apply(lambda x: round(x,1))
mean_obj

,n,BE1,BE2,BE3,BN,BP,RLHH,rank,min_idx,min_val
0,50,7735.6,5544.3,5972.1,6654.8,7541.8,5826.9,2,BE2,5544
1,75,12829.1,7537.7,11306.9,10039.0,11388.5,7995.2,2,BE2,7537
2,100,24252.4,9795.7,42836.6,17958.2,22096.5,9932.8,2,BE2,9795
3,150,38887.3,13596.7,49925.5,29004.7,37681.2,14552.0,2,BE2,13596
4,200,79302.6,37857.7,122671.1,42762.1,81441.4,17254.8,1,RLHH,17254


In [15]:
# mean_obj.to_csv('../result/paper/mean_objval.csv', index=False)

### 平均司机数

In [16]:
mean_obj = df_all[['n', 'method', 'driver']].groupby(by=['n', 'method'], as_index=False).mean()
mean_obj = concat_horizon(mean_obj, 'driver', indexes=['n'])
mean_obj = cal_best_method(mean_obj)
mean_obj['n'] = mean_obj['n'].astype('int')
mean_obj[methods] = mean_obj[methods].apply(lambda x: round(x,1))
mean_obj

,n,BE1,BE2,BE3,BN,BP,RLHH,rank,min_idx,min_val
0,50,15.6,14.0,14.3,14.8,15.4,14.1,4,BE2,13
1,75,22.1,18.2,21.0,20.1,21.0,18.6,2,BE2,18
2,100,35.4,25.2,48.4,31.1,33.9,25.4,2,BE2,25
3,150,54.9,37.1,62.7,48.1,54.1,37.6,2,BE2,37
4,200,115.1,85.3,145.7,89.4,116.6,71.8,1,RLHH,71


In [18]:
# mean_obj.to_csv('../result/paper/mean_driver.csv', index=False)

### 平均时间

In [19]:
# df_all = df_new.copy()
df_all['time'] = df_all['time'].apply(lambda x: min(x/60,60))   # 单位转为min

In [29]:
mean_time = df_all[['n', 'method', 'time']].groupby(by=['n', 'method'], as_index=False).mean()
mean_time = concat_horizon(mean_time, columns=['time'])
mean_time = cal_best_method(mean_time)
# mean_time[methods] = mean_obj[methods].apply(lambda x: round(x,1))  # astype(int)
mean_time

,n,BE1,BE2,BE3,BN,BP,RLHH,rank,min_idx,min_val
0,50.0,1.385679,0.658195,0.637735,0.557523,0.805847,0.245570,5,BE2,0
1,75.0,6.808897,3.558584,5.228397,4.115662,5.418405,1.882561,1,RLHH,1
2,100.0,10.388436,9.874236,10.316644,10.836041,10.102357,8.172861,1,RLHH,8
3,150.0,57.728885,60.000000,60.000000,60.000000,58.339035,57.920657,2,BE1,57
4,200.0,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,6,BE1,60


In [31]:
mean_time.to_csv('../result/paper/mean_time.csv', index=False)

## Figure

In [ ]:
colors = ['blue', 'brown', 'gray', 'green', 'orange', 'red']
markers = ['o', 'v', '^', '+', 'x', 's']
methods = {
    "BestEdges1": "BE1",
    "BestEdges2": "BE2",
    "BestEdges3": "BE3",
    "BestNodes": "BN",
    "BestPaths": "BP",
    "RLHH": "RLHH"
}

### 迭代过程

In [ ]:
def load_rlhh(rlhh_file):
    f = open(rlhh_file, 'r')
    lines = f.readlines()
    lines = [line.split("\n")[0] for line in lines]
    array = [[info.split(":")[1] for info in line.split(', ')] for line in lines]
    columns = [info.split(":")[0] for info in lines[0].split(', ')]
    df = pd.DataFrame(data=array, columns=columns)
    df[["reward", "objval", "time"]] = df[["reward", "objval", "time"]].astype('float64')
    return df[:-int(len(df)/2.5)]
    
def load_baseline(base_file):
    f = open(base_file, 'r')
    line = f.readline()   # 每次读取一行内容
    data = {}
    while line:
        if "heuristic" in line:
            method = line.split(":  ")[-1][:-1]
            objval = []
            time = []
        elif "iteration" in line:
            info = line[:-1].split(", ")
            objval.append(float(info[1]))
            time.append(float(info[2]))
        else:
            data[method] = {"objval": objval, "time": time}
        line = f.readline()
    return data

In [ ]:
def draw_iteration_process(ax, instance):
    base_file = f"../result/detail/log_{instance}.txt"
    rlhh_file = f"../result/detail/RLHH_log_{instance}.txt"
    data = load_baseline(base_file)
    df = load_rlhh(rlhh_file)
    for i, method in enumerate(data.keys()):
        info = data[method]
        if method in ["Hyper", "HH"]:
            continue
        ax.plot(info['time'], info['objval'], colors[i], label=methods[method])
    ax.plot(df['time'], df['objval'], 'red', label="RLHH")

    ax.set_xlabel("time")
    # plt.yticks([]) # 隐藏y坐标轴
    ax.axes.yaxis.set_ticklabels([]) # 隐藏y坐标轴
    ax.set_ylabel("objval")
    ax.legend()

In [ ]:
plt.rcParams['figure.figsize']=(15,3)
fig, axes = plt.subplots(nrows=1, ncols=4)
instances = ["50_01", "100_07", "150_10", "200_01"]
for i in range(4):
    draw_iteration_process(axes[i], instances[i])
    axes[i].set_title("vcsp_" + instances[i].split('_')[0] + "_01")
plt.savefig("F:/Users/Documents/汇报/img/vcsp_iteration_process.png", dpi=400, bbox_inches="tight")

## End